In [1340]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from itertools import combinations

In [1341]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1848,tt0146882,30000000,47126295,High Fidelity,John Cusack|Iben Hjejle|Todd Louiso|Jack Black...,Stephen Frears,"A comedy about fear of commitment, hating your...",When record store owner Rob Gordon gets dumped...,113,Comedy|Drama|Romance|Music,Buena Vista|Touchstone Pictures,3/17/2000,7.0,2000
1428,tt0395972,35000000,25224242,North Country,Charlize Theron|Elle Peterson|Thomas Curtis|Fr...,Niki Caro,All She Wanted Was To Make A Living. Instead S...,A fictionalized account of the first major suc...,126,Drama,Warner Bros.|Participant Media|Nick Wechsler P...,2/12/2005,6.8,2005
1635,tt0335438,60000000,170268750,Starsky & Hutch,Ben Stiller|Owen Wilson|Snoop Dogg|Fred Willia...,Todd Phillips,They're the man.,"Join uptight David Starsky and laid-back Ken ""...",101,Comedy|Crime,Weed Road Pictures|Warner Bros.|Dimension Film...,3/5/2004,5.5,2004
963,tt0303933,20000000,57588485,Drumline,Nick Cannon|Zoe Saldana|Orlando Jones|Leonard ...,Charles Stone III,Half time is game time,A fish-out-of-water comedy about a talented st...,118,Drama|Romance|Comedy|Music,Fox 2000 Pictures,12/13/2002,6.3,2002
196,tt2170593,13000000,54837234,St. Vincent,Bill Murray|Melissa McCarthy|Naomi Watts|Chris...,Theodore Melfi,Love Thy Neighbor,A young boy whose parents just divorced finds ...,102,Comedy,The Weinstein Company|Crescendo Productions|Ch...,10/9/2014,7.1,2014


In [1342]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [1343]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

def preparation_answer(incomming_data):
    movie_name = incomming_data.original_title.iloc[0]
    movie_id = incomming_data.imdb_id.iloc[0]
    print(movie_name, ' (', movie_id , ')', sep='')
    return



def year_detect(release_date):
    result = re.search(r'\d{4}', release_date)
    if result:
        return int(result.group(0))
    else:
        return 1970


def month_detect(release_date):
    result = re.search(r'^\d\d?', release_date)
    if result:
        month = int(result.group(0))
        if month < 1 or month > 12:
            month = 0
        return month
    else:
        return 0


def series_aggregation_first_column(incoming_data, agg_type='sum', top=5, sort_direction=False,):
    
    result_series = pd.Series(dtype=object)

    for column1, column2 in incoming_data.items():
        
        if agg_type == 'sum':
            agg_parametr = column2
        else:
            agg_parametr = 1
            
        for item in column1.split('|'):
            
            if item in result_series.index:
                result_series[item] += agg_parametr
            else:
                result_series[item] = agg_parametr

    return result_series.sort_values(ascending=sort_direction).head(top)
  

data['profit'] = data.revenue - data.budget


data['release_year'] = data.release_date.apply(year_detect)


data['release_month'] = data.release_date.apply(month_detect)


data['word_count_overiew'] = data.overview.apply(lambda s: len(re.findall(r'\w+', s.lower())))


In [1344]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,word_count_overiew
607,tt0795421,52000000,609841637,Mamma Mia!,Meryl Streep|Pierce Brosnan|Amanda Seyfried|Co...,Phyllida Lloyd,Take a trip down the aisle you'll never forget,"Set on an idyllic Greek island, the plot serve...",108,Comedy|Romance,Universal Pictures|Internationale Filmprodukti...,6/30/2008,6.2,2008,557841637,6,66
346,tt0810784,8500000,4440055,Bright Star,Abbie Cornish|Ben Whishaw|Paul Schneider|Kerry...,Jane Campion,First love burns brightest.,"In 1818, high-spirited young Fanny Brawne find...",119,Drama|Romance,BBC Films|PathÃ© Renn Productions|UK Film Coun...,11/5/2009,6.3,2009,-4059945,11,76
608,tt0421715,150000000,333932083,The Curious Case of Benjamin Button,Cate Blanchett|Brad Pitt|Tilda Swinton|Julia O...,David Fincher,"Life isn't measured in minutes, but in moments.","Tells the story of Benjamin Button, a man who ...",166,Fantasy|Drama|Thriller|Mystery|Romance,Paramount Pictures|Warner Bros.|Kennedy/Marsha...,11/24/2008,7.2,2008,183932083,11,15
1049,tt1716777,16000000,12434778,People Like Us,Chris Pine|Elizabeth Banks|Olivia Wilde|Michel...,Alex Kurtzman,Find your family.,"It tells the story of a struggling man who, af...",114,Drama,DreamWorks SKG|Touchstone Pictures,6/29/2012,6.6,2012,-3565222,6,55
92,tt2554274,55000000,74679822,Crimson Peak,Mia Wasikowska|Jessica Chastain|Tom Hiddleston...,Guillermo del Toro,Beware.,"In the aftermath of a family tragedy, an aspir...",119,Mystery|Thriller|Fantasy|Horror|Drama,Legendary Pictures,10/13/2015,6.4,2015,19679822,10,45


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [1345]:

movie_name = data[data.budget == data.budget.max()]

preparation_answer(movie_name)



Pirates of the Caribbean: On Stranger Tides (tt1298650)


In [1346]:
# +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [1347]:
# можно добавлять разные варианты решения

# TODO

# 2. Какой из фильмов самый длительный (в минутах)?

In [1348]:

movie_name = data[data.runtime == data.runtime.max()]

preparation_answer(movie_name)


Gods and Generals (tt0279111)


In [1349]:
# +
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [1350]:

movie_name = data[data.runtime == data.runtime.min()]

preparation_answer(movie_name)


Winnie the Pooh (tt1449283)


In [1351]:
# +
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [1352]:

round(data.runtime.mean())

110

In [1353]:
# +
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [1354]:

round(data.runtime.median())

107

In [1355]:
# +
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [1356]:
# лучше код получения столбца profit вынести в Предобработку что в начале

movie_name = data[data.profit == data.profit.max()]

preparation_answer(movie_name)

Avatar (tt0499549)


In [1357]:
# +
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [1358]:

movie_name = data[data.profit == data.profit.min()]

preparation_answer(movie_name)

The Lone Ranger (tt1210819)


In [1359]:
# +
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [1360]:
len(data[data.profit > 0])

1478

In [1361]:
# +
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [1362]:

movie_name = data[data.profit == data.query('release_year == 2008').profit.max()]

preparation_answer(movie_name)

The Dark Knight (tt0468569)


In [1363]:
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [1364]:

movie_name = data[data.profit == data.query('2012 <= release_year <= 2014').profit.min()]

preparation_answer(movie_name)

The Lone Ranger (tt1210819)


In [1365]:
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [1366]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

genre_series = pd.Series(data=[0], index=['none'])

for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        if genre in genre_series.index:
            genre_series[genre] += genre_count
        else:
            genre_series[genre] = genre_count

genre_series.sort_values(ascending=False).head(5)


Drama        782
Comedy       683
Thriller     596
Action       582
Adventure    415
dtype: int64

In [1367]:
answers['11'] = 'Drama'

ВАРИАНТ 2

можно сделать чере словарь..

In [1368]:
genre_counter = Counter()

for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        genre_counter[genre] += genre_count

genre_counter
# TODO - поиск максимального значения


Counter({'Comedy': 683,
         'Drama': 782,
         'Romance': 308,
         'Horror': 176,
         'Thriller': 596,
         'Action': 582,
         'Crime': 315,
         'Family': 260,
         'Animation': 139,
         'History': 62,
         'Science Fiction': 248,
         'Adventure': 415,
         'Fantasy': 222,
         'Mystery': 168,
         'Music': 64,
         'War': 58,
         'Documentary': 8,
         'Western': 19,
         'Foreign': 2})

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [1369]:

profitable_film_genre = data[data.profit > 0].groupby(['genres'])['profit'].count()

series_aggregation_first_column(profitable_film_genre, 'sum')


Drama        560
Comedy       551
Thriller     446
Action       444
Adventure    337
dtype: int64

In [1370]:
series_aggregation_first_column(profitable_film_genre, 'sum').index[0]

'Drama'

In [1371]:
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [1372]:
director_revenue = pd.Series(data=[0], index=['none'])

for director_all, revenue in data.groupby(['director'])['revenue'].sum().items():
    for director in director_all.split('|'):
        if director in director_revenue.index:
            director_revenue[director] += revenue
        else:
            director_revenue[director] = revenue

director_revenue.sort_values(ascending=False).head(3)

Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
dtype: int64

In [1373]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [1374]:

director_genres = data[ ['director','genres'] ][data.genres.str.contains('Action', na=False)]

director_genres = director_genres.groupby(['director'])['genres'].count()

series_aggregation_first_column(director_genres, 'sum', top=5)


Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
dtype: int64

In [1375]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [1376]:

top_actor_2012 = data.query('release_year == 2012').groupby(['cast'])['revenue'].sum()

series_aggregation_first_column( top_actor_2012 ,'sum', top=1)


Chris Hemsworth    2027450773
dtype: int64

In [1377]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [1378]:
# высокобюджетные - фильмы, где бюджет выше среднего по данной выборке

top_actor = data[data.budget > data.budget.mean()].groupby(['cast'])['revenue'].count()

series_aggregation_first_column( top_actor ,'sum', top=6)


Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Samuel L. Jackson    15
Eddie Murphy         15
Tom Cruise           15
dtype: int64

In [1379]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [1380]:
favorite_genre = data[data.cast.str.contains('Nicolas Cage', na=False)].groupby(['genres'])['cast'].count()

series_aggregation_first_column( favorite_genre ,'sum', top=5)

Action      17
Thriller    15
Drama       12
Crime       10
Fantasy      8
dtype: int64

In [1381]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [1382]:

movie_name = data[data.profit == data[data.production_companies.str.contains('Paramount Pictures')].profit.min()]

preparation_answer(movie_name)

K-19: The Widowmaker (tt0267626)


In [1383]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [1384]:
data.groupby(['release_year'])['revenue'].sum().sort_values().tail(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

In [1385]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [1386]:
wb_profit = data[data.production_companies.str.contains('Warner Bros', na=False)]

wb_profit.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).head(1)

release_year
2014    2295464519
Name: profit, dtype: int64

In [1387]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [1388]:
# Вариант 1
data.release_month.value_counts().sort_values().tail(1)

9    227
Name: release_month, dtype: int64

In [1389]:
# Вариант 2
data.groupby(['release_month'])['imdb_id'].count().sort_values().tail(1)

release_month
9    227
Name: imdb_id, dtype: int64

In [1390]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [1391]:
# data.query('6<=release_month<=8').shape
# len(data.query('6 <= release_month <= 8'))

len(data.query('release_month in [6, 7, 8]'))

450

In [1392]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [1393]:

winter_director = data.query('release_month in [1, 2, 12]').director.value_counts()

series_aggregation_first_column(winter_director,'sum', top=3)

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
dtype: int64

In [1394]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [1395]:

max_len = data.original_title.str.len().max()

data[data.original_title.str.len() == max_len].production_companies.iloc[0].split('|')[1]



'Four By Two Productions'

In [1396]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['24'] = 'Four By Two Productions' 

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [1397]:
# production_companies_result = pd.DataFrame(columns=['production_companies', 'overview_word_count'])

# prod_companies = data[['production_companies', 'overview']].head(50)

# for i in prod_companies.index:
#     companies = prod_companies.iloc[i].production_companies
#     overview = prod_companies.iloc[i].overview
#     word_count = len(re.findall(r'\w+', overview.lower()))
    
#     for prod_companie in companies.split('|'):
        
#         new_location = len(production_companies_result)
        
#         production_companies_result.loc[new_location] = [prod_companie, word_count]


# production_companies_result.groupby(['production_companies'])['overview_word_count'].max()

# production_companies_result.production_companies.value_counts()
# production_companies_result.overview_word_count.max()

In [1398]:


word_count_overiew_max = data[data.word_count_overiew > data.word_count_overiew.mean()].word_count_overiew.max()

data[data.word_count_overiew == word_count_overiew_max].production_companies.iloc[0].split('|')[3]


'Midnight Picture Show'

In [1399]:
# +
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [1400]:
count_last_procent = round(len(data)*0.01)

last_procent = data.vote_average.sort_values(ascending=False).head(count_last_procent).index

data.loc[last_procent][['original_title','vote_average']]


,original_title,vote_average
599,The Dark Knight,8.1
125,The Imitation Game,8.0
9,Inside Out,8.0
118,Interstellar,8.0
34,Room,8.0
1081,The Lord of the Rings: The Return of the King,7.9
138,The Grand Budapest Hotel,7.9
1183,The Wolf of Wall Street,7.9
370,Inception,7.9
119,Guardians of the Galaxy,7.9


In [1401]:
# Проверка наличия 3-х фильмов в выборке 1% лучших фильмов

#     'BloodRayne, The Adventures of Rocky & Bullwinkle',
#     'Batman Begins, The Lord of the Rings: The Return of the King, Upside Down',
#     '300, Lucky Number Slevin, Kill Bill: Vol. 1',
#     'Upside Down, Inside Out, Iron Man']


top_movies = ['Inside Out','The Dark Knight','12 Years a Slave']

data.loc[last_procent].query('original_title in @top_movies')[['original_title','vote_average']]

data.query('original_title in @top_movies')[['original_title','vote_average']]

,original_title,vote_average
9,Inside Out,8.0
599,The Dark Knight,8.1
1191,12 Years a Slave,7.9


In [1402]:
# 
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [1403]:
# 
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 2

In [1404]:
pair_actors = [  'Johnny Depp & Helena Bonham Carter',
                'Ben Stiller & Owen Wilson',
                'Vin Diesel & Paul Walker',
                'Adam Sandler & Kevin James',
                'Daniel Radcliffe & Rupert Grint']

actor_pair_work = pd.Series(dtype=object)

for pair_actor in pair_actors:
    actor1, actor2 =  pair_actor.split(' & ')
    pair_work_count = data[data.cast.str.contains(actor1) & data.cast.str.contains(actor2)].imdb_id.count()
    #print(pair_work_count, '|', pair_actor)
    actor_pair_work[pair_actor] = pair_work_count

actor_pair_work.sort_values(ascending=False).head(1).index[0]


'Daniel Radcliffe & Rupert Grint'

# Submission

In [1405]:
# в конце можно посмотреть свои ответы к каждому вопросу

# Вопрос № 1 ........ [ + ]
# Вопрос № 2 ........ [ + ]
# Вопрос № 3 ........ [ + ]
# Вопрос № 4 ........ [ + ]
# Вопрос № 5 ........ [ + ]
# Вопрос № 6 ........ [ + ]
# Вопрос № 7 ........ [ + ]
# Вопрос № 8 ........ [ + ]
# Вопрос № 9 ........ [ + ]
# Вопрос № 10 ....... [ + ]
# Вопрос № 11 ....... [ + ]
# Вопрос № 12 ....... [ + ]
# Вопрос № 13 ....... [ + ]
# Вопрос № 14 ....... [ + ]
# Вопрос № 15 ....... [ + ]
# Вопрос № 16 ....... [ + ]
# Вопрос № 17 ....... [ + ]
# Вопрос № 18 ....... [ + ]
# Вопрос № 19 ....... [ + ]
# Вопрос № 20 ....... [ + ]
# Вопрос № 21 ....... [ + ]
# Вопрос № 22 ....... [ + ]
# Вопрос № 23 ....... [ + ]
# Вопрос № 24 ....... [ + ]
# Вопрос № 25 ....... [ + ]
# Вопрос № 26 ....... [ + ]
# Вопрос № 27 ....... [ + ]

answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [1406]:
# и убедиться что ни чего не пропустил)
len(answers)

27

### Спасибо за внимание!